Natural Language Processing with Python – Analyzing Text with the Natural Language Toolkit Steven Bird, mEwan Klein, and Edward Loper http://www.nltk.org/book/

# Chapter 08 - Analyzing Sentence Structure

## 8.1 Some Grammatical Dilemmas

### Linguistic Data and Unlimited Possibilities

In this chapter, we will adopt the formal framework of “generative grammar,” in which a “language” is considered to be nothing more than an enormous collection of all grammatical sentences, and a grammar is a formal notation that can be used for “generating” the members of this set. Grammars use recursive productions of the form S → S and S, as we will explore in Section 8.3. In Chapter 10 we will extend this, to
automatically build up the meaning of a sentence out of the meanings of its parts.

### Ubiquitous Ambiguity

Let’s take a closer look at the ambiguity in the phrase: I shot an elephant in my pajamas.First we need to define a simple grammar:

In [13]:
groucho_grammar = nltk.CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> Det N | Det N PP | 'I'
    VP -> V NP | VP PP
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")

In [14]:
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

In [15]:
parser = nltk.ChartParser(groucho_grammar)

In [16]:
for tree in parser.parse(sent):
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


## 8.2 What's the Use of Syntax?

### Beyond n-grams

##  8.3 Context-Free Grammar

### A Simple Grammar

Let’s start off by looking at a simple context-free grammar (CFG). By convention, the lefthand side of the first production is the start-symbol of the grammar, typically S, and all well-formed trees must have this symbol as their root label. In NLTK, contextfree grammars are defined in the nltk.grammar module.

In [18]:
grammar1 = nltk.CFG.fromstring("""
    S -> NP VP
    VP -> V NP | V NP PP
    PP -> P NP
    V -> "saw" | "ate" | "walked"
    NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
    Det -> "a" | "an" | "the" | "my"
    N -> "man" | "dog" | "cat" | "telescope" | "park"
    P -> "in" | "on" | "by" | "with"
""")

In [19]:
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


In [20]:
nltk.app.rdparser()

### Writing Your Own Grammars

If you are interested in experimenting with writing CFGs, you will find it helpful to create and edit your grammar in a text file, say, mygrammar.cfg. You can then load it into NLTK and parse with it as follows:

In [34]:
%%file mygrammar.cfg
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> "saw" | "ate" | "walked"
NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
Det -> "a" | "an" | "the" | "my"
N -> "man" | "dog" | "cat" | "telescope" | "park"
P -> "in" | "on" | "by" | "with"

Writing mygrammar.cfg


In [35]:
grammar1 = nltk.data.load('file:mygrammar.cfg')
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


In [36]:
!del /f mygrammar.cfg

### Recursion in Syntactic Structure

The production Nom -> Adj Nom (where Nom is the category of nominals) involves direct recursion on the category Nom, whereas indirect recursion on S arises from the combination of two productions, namely S -> NP VP and VP -> V S.

In [42]:
grammar2 = nltk.CFG.fromstring("""
    S  -> NP VP
    NP -> Det Nom | PropN
    Nom -> Adj Nom | N
    VP -> V Adj | V NP | V S | V NP PP
    PP -> P NP
    PropN -> 'Buster' | 'Chatterer' | 'Joe'
    Det -> 'the' | 'a'
    N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
    Adj  -> 'angry' | 'frightened' |  'little' | 'tall'
    V ->  'chased'  | 'saw' | 'said' | 'thought' | 'was' | 'put'
    P -> 'on'
""")

In [43]:
rd_parser = nltk.RecursiveDescentParser(grammar2)

In [44]:
sent = "the angry bear chased the frightened little squirrel".split()
for tree in rd_parser.parse(sent):
    tree.draw()

In [46]:
sent = "Chatterer said Buster thought the tree was tall".split()
for tree in rd_parser.parse(sent):
    tree.draw()

## 8.4 Parsing with Context-Free Grammar

A parser processes input sentences according to the productions of a grammar, and builds one or more constituent structures that conform to the grammar. A grammar is a declarative specification of well-formedness—it is actually just a string, not a program.
A parser is a procedural interpretation of the grammar. It searches through the space of trees licensed by a grammar to find one that has the required sentence along its fringe.

### Recursive Descent Parsing

The simplest kind of parser interprets a grammar as a specification of how to break a high-level goal into several lower-level subgoals. The top-level goal is to find an S. The S → NP VP production permits the parser to replace this goal with two subgoals: find an NP, then find a VP. Each of these subgoals can be replaced in turn by sub-subgoals, using productions that have NP and VP on their lefthand side. Eventually, this expansion process leads to subgoals such as: find the word telescope.

In [104]:
rd_parser = nltk.RecursiveDescentParser(grammar1)
sent = 'Mary saw a dog'.split()
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


Recursive descent parsing has **three key shortcomings**.
1. First, left-recursive productions like NP -> NP PP send it into an infinite loop.
2. Second, the parser wastes a lot of time considering words and structures that do not correspond to the input sentence.
3. Third, the backtracking process may discard parsed constituents that will need to be rebuilt again later.  For example, backtracking over VP -> V NP will discard the subtree created for the NP. If the parser then proceeds with VP -> V NP PP, then the NP subtree must be created all over again.

Recursive descent parsing is a kind of **top-down** parsing. Top-down parsers use a grammar to predict what the input will be, before inspecting the input! However, since the input is available to the parser all along, it would be more sensible to consider the input sentence from the very beginning. This approach is called **bottom-up** parsing, and we will see an example in the next section.

### Shift-Reduce Parsing

A simple kind of bottom-up parser is the shift-reduce parser. In common with all bottom-up parsers, a shift-reduce parser tries to find sequences of words and phrases that correspond to the righthand side of a grammar production, and replace them with the lefthand side, until the whole sentence is reduced to an S.

In [ ]:
nltk.app.srparser()

In [110]:
sr_parser = nltk.ShiftReduceParser(grammar1)
sent = 'Mary saw a dog'.split()
for tree in sr_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


**A shift-reduce parser can reach a dead end and fail to find any parse, even if the input sentence is well-formed according to the grammar**. When this happens, no input remains, and the stack contains items which cannot be reduced to an S. The problem arises because there are choices made earlier that cannot be undone by the parser (although users of the graphical demonstration can undo their choices). There are two kinds of choices to be made by the parser:<div>
(a) which reduction to do when more than one is possible.<div>
(b) whether to shift or reduce when either action is possible.

**A shift-reduce parser may be extended to implement policies for resolving such conflicts**. For example, it may address shift-reduce conflicts by shifting only when no reductions are possible, and it may address reduce-reduce conflicts by favoring the reduction operation that removes the most items from the stack. (A generalization of shift-reduce parser, a "lookahead LR parser", is commonly used in programming language compilers.)

**Advantages:**

1. The advantage of shift-reduce parsers over recursive descent parsers is that they only build structure that corresponds to the words in the input.
2. Furthermore, they only build each sub-structure once, e.g. NP(Det(the), N(man)) is only built and pushed onto the stack a single time, regardless of whether it will later be used by the VP -> V NP PP reduction or the NP -> NP PP reduction.

### Well-Formed Substring Tables

The simple parsers discussed in the previous sections suffer from limitations in both completeness and efficiency. In order to remedy these, we will apply the algorithm design technique of dynamic programming to the parsing problem.

In [ ]:
text = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

In [ ]:
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i+1] = productions[0].lhs()
    return wfst

In [ ]:
def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end] = index[(nt1,nt2)]
                    if trace:
                        print "[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % \
                        (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end)
    return wfst

In [ ]:
def display(wfst, tokens):
    print '\nWFST ' + ' '.join([("%-4d" % i) for i in range(1, len(wfst))])
    for i in range(len(wfst)-1):
        print "%d " % i,
        for j in range(1, len(wfst)):
            print "%-4s" % (wfst[i][j] or '.'),
        print

In [ ]:
tokens = "I shot an elephant in my pajamas".split()

In [ ]:
wfst0 = init_wfst(tokens, groucho_grammar)

In [ ]:
display(wfst0, tokens)

In [ ]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)

In [ ]:
display(wfst1, tokens)

In [ ]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)

# Dependencies and Dependency Grammar

Phrase structure grammar is concerned with how words and sequences of words combine
to form constituents. A distinct and complementary approach, dependency
grammar, focuses instead on how words relate to other words. Dependency is a binary
asymmetric relation that holds between a head and its dependents. The head of a
sentence is usually taken to be the tensed verb, and every other word is either dependent
on the sentence head or connects to it through a path of dependencies.

In [ ]:
groucho_dep_grammar = nltk.parse_dependency_grammar("""
    'shot' -> 'I' | 'elephant' | 'in'
    'elephant' -> 'an' | 'in'
    'in' -> 'pajamas'
    """)

In [ ]:
print groucho_dep_grammar

In [ ]:
pdp = nltk.ProjectiveDependencyParser(groucho_dep_grammar)

In [ ]:
sent = 'I shot an elephant in my pajamas'.split()

In [ ]:
trees = pdp.parse(sent)

In [ ]:
for tree in trees:
    print tree

# Grammar Development

Parsing builds trees over sentences, according to a phrase structure grammar. Now, all
the examples we gave earlier only involved toy grammars containing a handful of productions.

## Treebanks and Grammars

The corpus module defines the treebank corpus reader, which contains a 10% sample
of the Penn Treebank Corpus.

In [ ]:
from nltk.corpus import treebank

In [ ]:
t = treebank.parsed_sents('wsj_0001.mrg')[0]

In [ ]:
print t

In [ ]:
def filter(tree):
    child_nodes = [child.node for child in tree
        if isinstance(child, nltk.Tree)]
    return (tree.node == 'VP') and ('S' in child_nodes)

In [ ]:
from nltk.corpus import treebank

In [ ]:
[subtree for tree in treebank.parsed_sents()
         for subtree in tree.subtrees(filter)]

In [ ]:
entries = nltk.corpus.ppattach.attachments('training')

In [ ]:
table = nltk.defaultdict(lambda: nltk.defaultdict(set))

In [ ]:
for entry in entries:
    key = entry.noun1 + '-' + entry.prep + '-' + entry.noun2
    table[key][entry.attachment].add(entry.verb)

In [ ]:
for key in sorted(table):
    if len(table[key]) > 1:
        print key, 'N:', sorted(table[key]['N']), 'V:', sorted(table[key]['V'])

In [ ]:
nltk.corpus.sinica_treebank.parsed_sents()[3450].draw()

## Pernicious Ambiguity

Unfortunately, as the coverage of the grammar increases and the length of the input
sentences grows, the number of parse trees grows rapidly. In fact, it grows at an astronomical
rate.

In [ ]:
grammar = nltk.parse_cfg("""
                        S -> NP V NP
                        NP -> NP Sbar
                        Sbar -> NP V
                        NP -> 'fish'
                        V -> 'fish'
                        """)

In [ ]:
tokens = ["fish"] * 5

In [ ]:
cp = nltk.ChartParser(grammar)

In [ ]:
for tree in cp.nbest_parse(tokens):
    print tree

## Weighted Grammar

As we have just seen, dealing with ambiguity is a key challenge in developing broadcoverage
parsers. Chart parsers improve the efficiency of computing multiple parses of
the same sentences, but they are still overwhelmed by the sheer number of possible
parses. Weighted grammars and probabilistic parsing algorithms have provided an effective
solution to these problems.

In [ ]:
def give(t):
    return t.node == 'VP' and len(t) > 2 and t[1].node == 'NP'\
            and (t[2].node == 'PP-DTV' or t[2].node == 'NP')\
            and ('give' in t[0].leaves() or 'gave' in t[0].leaves())

In [ ]:
def sent(t):
    return ' '.join(token for token in t.leaves() if token[0] not in '*-0')

In [ ]:
def print_node(t, width):
        output = "%s %s: %s / %s: %s" %\
            (sent(t[0]), t[1].node, sent(t[1]), t[2].node, sent(t[2]))
        if len(output) > width:
            output = output[:width] + "..."
        print output

In [ ]:
for tree in nltk.corpus.treebank.parsed_sents():
    for t in tree.subtrees(give):
        print_node(t, 72)

In [ ]:
grammar = nltk.parse_pcfg("""
                        S -> NP VP [1.0]
                        VP -> TV NP [0.4]
                        VP -> IV [0.3]
                        VP -> DatV NP NP [0.3]
                        TV -> 'saw' [1.0]
                        IV -> 'ate' [1.0]
                        DatV -> 'gave' [1.0]
                        NP -> 'telescopes' [0.8]
                        NP -> 'Jack' [0.2]
                        """)

In [ ]:
print grammar

In [ ]:
viterbi_parser = nltk.ViterbiParser(grammar)

In [ ]:
print viterbi_parser.parse(['Jack', 'saw', 'telescopes'])